In [1]:
from imutils import paths
import cv2
import os
import pickle
from recognition_modules.pre_process.pre_process import pre_process_frame
from recognition_modules.feature_extractor.ageitgey_model import get_embeddings
from recognition_modules.face_match.election_based import get_name_election

In [2]:
with open('encodings_clahe_face.pickle', 'rb') as handle:
    data = pickle.load(handle)

In [3]:
# load paths
paths = list(paths.list_images("Train/"))
# load detector
detector = cv2.CascadeClassifier('detector.xml')

In [4]:
results = {
    'predict':[],
    'real':[],
    'tolerance':[]
}

In [5]:
def improve_contrast_image_using_clahe(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv_planes = cv2.split(hsv)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    hsv_planes[2] = clahe.apply(hsv_planes[2])
    hsv = cv2.merge(hsv_planes)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

In [6]:
for i in range(40,71,5):
    print(f'Info: Tolerancia {i/100}')
    for path in paths:
        try:
            # get the frame
            frame = cv2.imread(path)
            # pre process frame
            rects = pre_process_frame(frame,detector)
            x1, y1, width, height = rects
            x2, y2 = x1 + width, y1 + height
            face = frame[y1:y2, x1:x2]
            face = improve_contrast_image_using_clahe(face)
            # get embeddings
            encoding = get_embeddings(rects,face)
            # get tha name
            name = get_name_election(encoding,data,input_tolerance= (i/100))
            # save in dictionary
            results['predict'].append(name)
            results['real'].append(path.split(os.path.sep)[-2])
            results['tolerance'].append((i/100))
        except:
            pass

Info: Tolerancia 0.4
Info: Tolerancia 0.45
Info: Tolerancia 0.5
Info: Tolerancia 0.55
Info: Tolerancia 0.6
Info: Tolerancia 0.65
Info: Tolerancia 0.7


In [7]:
import pandas as pd

In [8]:
data = pd.DataFrame(results)

In [9]:
data['result'] = data['predict'] == data['real']

In [23]:
with open('clahe_frame.pickle', 'wb') as handle:
    pickle.dump(results, handle)